In [26]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier

data = pd.read_csv('train_jqd04QH.csv')
test = pd.read_csv('test_GYi4Gz5.csv')

# #Replacing Null values of Company_type to Other
data['company_type'].fillna('NpO', inplace = True)
test['company_type'].fillna('NpO', inplace = True)

#replacing with medium company size
data['company_size'].fillna('50-99', inplace  = True)
test['company_size'].fillna('50-99', inplace  = True)

# # #Managing all values accordingly
data.experience[data.experience == '>20'] = 20 
data.experience[data.experience == '<1'] = 1 

test.experience[test.experience == '>20'] = 20 
test.experience[test.experience == '<1'] = 1 

# #Replacing NULL values in experience with median
data['experience'].fillna(9, inplace = True)
test['experience'].fillna(9, inplace = True)

# #Replacing all values in major decipline with 'Other
data['major_discipline'].fillna('Other', inplace = True)
test['major_discipline'].fillna('Other', inplace = True)

# #Handling all the values 
data.last_new_job[data.last_new_job == '>4'] = 5 
test.last_new_job[test.last_new_job == '>4'] = 5 

# #Replscing all the NULL values with Mode
data['last_new_job'].fillna(2, inplace = True)
test['last_new_job'].fillna(2, inplace = True)

# #Replacing all the NULL values in education_evel with mode
data['education_level'].fillna('Masters', inplace = True)
test['education_level'].fillna('Masters', inplace = True) 

# #Replacing all the NULL values in enrolled university with mode
data['enrolled_university'].fillna('Part time course', inplace = True)
test['enrolled_university'].fillna('Part time course', inplace = True)

# #Filling NULL values with mode
data.gender.fillna('Other', inplace =  True) #data['gender'].mode()[0]
test.gender.fillna('Other', inplace =  True) #test['gender'].mode()[0]

X = data.drop(['target'] ,axis = 1)
y = data['target'].copy()
X_test = test

            
'''

X = pd.concat([X.get(['enrollee_id','city','city_development_index', 'gender', 'relevent_experience', 'experience',
                                                                            'last_new_job', 'training_hours' ]),
                           pd.get_dummies(X['enrolled_university'], prefix='enrolled_university'),
                           pd.get_dummies(X['education_level'], prefix='education_level'),
                           pd.get_dummies(X['major_discipline'], prefix='major_discipline'),
                           pd.get_dummies(X['company_size'], prefix='company_size'),
                           pd.get_dummies(X['company_type'], prefix='company_type')],axis=1)

X_test = pd.concat([X_test.get(['enrollee_id','city','city_development_index', 'gender', 'relevent_experience', 'experience',
                                                                            'last_new_job', 'training_hours' ]),
                           pd.get_dummies(X_test['enrolled_university'], prefix='enrolled_university'),
                           pd.get_dummies(X_test['education_level'], prefix='education_level'),
                           pd.get_dummies(X_test['major_discipline'], prefix='major_discipline'),
                           pd.get_dummies(X_test['company_size'], prefix='company_size'),
                           pd.get_dummies(X_test['company_type'], prefix='company_type')],axis=1)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=42)
'''

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\admin\appdata\local\programs\python\python36\lib\site-p

"\n\nX = pd.concat([X.get(['enrollee_id','city','city_development_index', 'gender', 'relevent_experience', 'experience',\n                                                                            'last_new_job', 'training_hours' ]),\n                           pd.get_dummies(X['enrolled_university'], prefix='enrolled_university'),\n                           pd.get_dummies(X['education_level'], prefix='education_level'),\n                           pd.get_dummies(X['major_discipline'], prefix='major_discipline'),\n                           pd.get_dummies(X['company_size'], prefix='company_size'),\n                           pd.get_dummies(X['company_type'], prefix='company_type')],axis=1)\n\nX_test = pd.concat([X_test.get(['enrollee_id','city','city_development_index', 'gender', 'relevent_experience', 'experience',\n                                                                            'last_new_job', 'training_hours' ]),\n                           pd.get_dummies(X_test['enrol

In [27]:
X.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
dtype: object

In [29]:

from sklearn import preprocessing

for i in X.columns:
    if X[i].dtype == 'object':
        if len(list(X[i].unique())) >=2:
            #cols.append(i)
            le = preprocessing.LabelEncoder()
            le.fit(X[i].astype(str))
            X[i] = le.transform(X[i].astype(str))
            X_test[i] = le.transform(X_test[i].astype(str))
            #X_val[i] =   le.transform(X_val[i].astype(str))
            #X_test[i] =  le.transform(X_test[i].astype(str))

# catboost

In [30]:
X.columns

Index(['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours'],
      dtype='object')

In [31]:
fea = [5,6,7,9,10]

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=42)

In [33]:
from catboost import CatBoostClassifier


In [34]:
cb_model = CatBoostClassifier(iterations=1000,
                              learning_rate=0.1,
                              depth=7,
                              l2_leaf_reg=40,
                              bootstrap_type='Bernoulli',
                              subsample=0.7,
                              scale_pos_weight=5,
                              eval_metric='AUC',
                              metric_period=50,
                              od_type='Iter',
                              od_wait=45,
                              random_seed=17,
                              allow_writing_files=False)


In [35]:
X.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,23798,41,0.689,1,0,2,0,5,13,1,6,0,106
1,29166,112,0.923,1,0,2,0,5,5,7,1,0,69
2,46,48,0.910,2,0,2,0,5,16,4,5,1,4
3,18527,93,0.666,1,0,2,0,5,5,4,6,0,26
4,21751,2,0.887,2,1,2,2,5,18,4,3,1,88


In [42]:
cb_model.fit(X_train, y_train,cat_features=fea,
             eval_set=(X_val, y_val),
            
             
             verbose=True)

0:	learn: 0.6419945	test: 0.6058316	best: 0.6058316 (0)	total: 105ms	remaining: 1m 44s
50:	learn: 0.6893470	test: 0.6281109	best: 0.6306283 (47)	total: 3.66s	remaining: 1m 8s
Stopped by overfitting detector  (45 iterations wait)

bestTest = 0.6306282755
bestIteration = 47

Shrink model to first 48 iterations.


In [43]:
sub = pd.read_csv('sample_submission_sxfcbdx.csv')

In [44]:
pred = cb_model.predict_proba(X_test)

In [45]:
pred

array([[0.28056268, 0.71943732],
       [0.66790141, 0.33209859],
       [0.35059213, 0.64940787],
       ...,
       [0.72603252, 0.27396748],
       [0.31274078, 0.68725922],
       [0.43741829, 0.56258171]])

In [46]:
sub['target'] = pred[:, 1]

In [47]:
sub.to_csv('cat.csv', index = False)